In [ ]:
import torch
import torch.nn as nn
import statistics
import torchvision.models as models ## used to load the pytorch models for vision

## We define a module that link our CNN to RNN model.

In [ ]:
class CNNtoRNN(nn.Module): # cnn to rnn is hooked here.
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size)
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions)
        return outputs
    
    # In decoder rnn we are making some target caption for every timestep.The predictions are used in our loss function. 
    # Actual predictions are made from the dataset.

    def caption_image(self, image, vocabulary, max_length=50): # this is used so that we can use this to get predicted word.
        result_caption = [] 

        with torch.no_grad(): 
            x = self.encoderCNN(image).unsqueeze(0) # we take imgae and send to encodder cnn a dimmension for batch.
            states = None # hidden and cell states for lstm.

            for _ in range(max_length): ## upto the max_length words prediction, 50 here and depends on dataset.
                hiddens, states = self.decoderRNN.lstm(x, states)  # at beginning it will be initialized as 0.
                output = self.decoderRNN.linear(hiddens.squeeze(0)) # sending single image and unsquence for that batch dimension. 
                predicted = output.argmax(1) # so we taking word with higgest probablity.
                result_caption.append(predicted.item()) 
                x = self.decoderRNN.embed(predicted).unsqueeze(0) # taking the predicted word.
                # now x is looped through and send as next word in RNN.

                if vocabulary.itos[predicted.item()] == "<EOS>": # check if vocab is equal to end of sentence then break.
                    break

        return [vocabulary.itos[idx] for idx in result_caption] # at end we are returning the string and not just the indices,
                                                                #                           that correspond to the presictions.